In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Get results of experiment testing impact of contamination percentage on performance
df = pd.read_json("../reports/contamination.jsonl", lines=True, orient="records")

df["preds"] = df["preds"].apply(np.array)
df["labels"] = df["labels"].apply(np.array)
df["is_anom"] = df["is_anom"].apply(np.array)

df["accuracy"] = df.apply(
    lambda row: (row["preds"] == row["labels"])[~row["is_anom"]].mean() * 100, axis=1
)
parameters = ["dataset", "anomaly_type", "p_anomaly", "len_anomaly"]

dfm = df.groupby(parameters)["accuracy"].agg(["mean", "std"])
dfm = dfm.reset_index()

In [ ]:
dfm[(dfm['anomaly_type'] == 'feature_swap') & (dfm['len_anomaly'] == 2)]

In [ ]:
# Plot error rate vs. anomaly percentage
fig, axs = plt.subplots(nrows=3, figsize=(6, 8), sharex=True, sharey=True)

for ax, (anom_type, df_anom_type) in zip(axs, dfm.groupby("anomaly_type")):
    ax.set_title(anom_type)
    for len_anomaly, df_len_anomaly in df_anom_type.groupby("len_anomaly"):
        ax.plot(
            df_len_anomaly["p_anomaly"], 100 - df_len_anomaly["mean"], label=len_anomaly
        )


ax.legend()
fig.tight_layout()